In [28]:
%reset

In [1]:
# 加载所需的库
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
def initialization(parameter):
    #产生随机的工作表
    joblist = pd.DataFrame(columns=['d_min','d_max','w_o','w_f','p','w','start','stop'],
                           dtype = int)
    #Trapezoidal Suitability Function，打分函数，列标题代表区块开始的时间点
    tsf = pd.DataFrame(0,
                       index=range(parameter['time_units']),
                       columns=range(parameter['n']))
    # 依次生成每一个任务的属性
    for i in range(parameter['n']):
        #时间段：w,d_min,d_max,d
        d_min = np.random.randint(parameter['d_min'],parameter['d_max']+1)
        d_max = np.random.randint(d_min,parameter['d_max']+1)
        w = np.random.randint(d_max,parameter['w_max']+1)
        #时间点：w_o,w_f,start,stop
        w_o = np.random.randint(0,parameter['time_units']-w)
        w_f = w_o + w
        #优先级：p
        p = np.random.randint(1,parameter['p_max']+1)
        # 更新joblist
        joblist.loc[i] = (d_min,d_max,w_o,w_f,p,w,0,0)

        # 更新打分函数
        s_i = tsf[i].copy()
        # 梯形4个顶点
        a = w_o
        b = int( w_o + np.ceil((w - d_max)/2) )
        c = b + d_max
        d = w_f
        #赋值1
        s_i[range(b,c)] = 1
        #赋值2
        if b > a:
            degree_1 = 1/(2*(b-a))
            n_1 = 1
            for j in range(a,b):
                s_i[j] = degree_1 * (2 * n_1 - 1)
                n_1 += 1
        #赋值3
        if d > c:
            degree_2 = 1/(2*(d-c))
            n_2 = 1
            for j in range(d-1,c-1,-1):
                s_i[j] = degree_2 * (2 * n_2 - 1)
                n_2 += 1
        #更新打分函数
        tsf[i] = s_i

    return tsf,joblist

In [3]:
def pd_sequencing(parameter,pd_jobindex,job):
    pd_job = job.copy()
    #总时间轴,1代表位置可用
    pd_timetable = pd.Series(1,index=range(parameter['time_units']))
    #初始化总时间轴，因为有可能已经有任务指定了时间
    for i in range(parameter['n']):
        pd_timetable.loc[range(pd_job.loc[i,'start'],pd_job.loc[i,'stop'])]=0
    #任务时间轴,1代表该时间在该任务的窗口中
    empty_timetable = pd.Series(0,index=range(parameter['time_units']))
    #建立空工作表，按顺序存放被选择的工作的序号
    chosen_job_list = list()

    #对每一个工作进行判断
    for i in range(len(pd_jobindex)):
        job_index = pd_jobindex[i]
        job_info = pd_job.loc[job_index,]

        d_min = job_info.loc['d_min']
        d_max = job_info.loc['d_max']
        w_o = job_info.loc['w_o']
        w_f = job_info.loc['w_f']
        p = job_info.loc['p']
        w = job_info.loc['w']
        #下序列用来存放每一个开始时间，长度为d_min的工作带来的收益
        position_value = pd.Series(0,index=range(parameter['time_units']))
        #假设没有可供使用的位置
        at_least_one = False
        #对每一个位置进行测试
        for j in range(w-d_min+1):
            test_start = w_o+j
            test_stop = test_start+d_min
            test_job = empty_timetable.copy()
            test_job.loc[range(test_start,test_stop)]=1
            #判断位置是否可用
            if np.sum(test_job*pd_timetable) == d_min:
                at_least_one = True
                position_value.loc[test_start] =\
                    np.sum(TSF.loc[range(test_start,test_stop),job_index])
        #如果有至少一个可用位置，则选择最大的那个
        if at_least_one:
            decide_start = \
                np.min(position_value.index[position_value == position_value.max()])
            decide_stop = decide_start+d_min
            #更新工作表
            pd_job.loc[job_index,'start'] = decide_start
            pd_job.loc[job_index,'stop'] = decide_stop
            #更新工作序号表
            chosen_job_list.append(job_index)
        else:
            decide_start = 0
            decide_stop = 0
        #刚刚被占用的位置不再可用
        pd_timetable.loc[range(decide_start,decide_stop)]=0

    return pd_job,chosen_job_list

In [4]:
def pd_timing(pd_chosen_jobindex,pd_job):
    test_pd_job = pd_job.copy()
    #对每一个工作进行左右延伸
    for i in range(len(pd_chosen_jobindex)):
        job_index = pd_chosen_jobindex[i]
        job_info = test_pd_job.loc[job_index,]

        d_min = job_info.loc['d_min']
        d_max = job_info.loc['d_max']
        w_o = job_info.loc['w_o']
        w_f = job_info.loc['w_f']
        p = job_info.loc['p']
        w = job_info.loc['w']
        start = job_info.loc['start']
        stop = job_info.loc['stop']
        #左右边界
        new_start = np.nanmax([test_pd_job.loc[test_pd_job.loc[:,'stop']<=start,'stop'].max(),
                               w_o,stop-d_max])
        new_stop = np.nanmin([test_pd_job.loc[test_pd_job.loc[:,'start']>=stop,'start'].min(),
                              w_f,new_start+d_max])
        #左右延伸
        test_pd_job.loc[job_index,'start'] = new_start
        test_pd_job.loc[job_index,'stop'] = new_stop
    return test_pd_job

In [5]:
def get_timetable(job,parameter):
    empty_timetable = pd.Series(-1,index=range(parameter['time_units']))
    for i in range(parameter['n']):
        empty_timetable.loc[range(job.loc[i,'start'],job.loc[i,'stop'])]=i
    return empty_timetable

In [6]:
def score(JOB,TSF,parameter):
    s = 0
    for i in range(parameter['n']):
        start = JOB.loc[i,'start']
        stop = JOB.loc[i,'stop']
        p = JOB.loc[i,'p']

        s_i = 0
        for j in range(start,stop):
            s_i += TSF.loc[j,i]

        s += s_i * p
    return s

In [7]:
# 设置参数
parameter = {'n':40,
             'time_units' : 100,
             'w_max':25,
             'd_min':1,
             'd_max':25,
             'p_max':10}

In [8]:
#初始化
np.random.seed(1)
TSF,JOB = initialization(parameter)
job_index_ordered_by_p = JOB.sort_values(by='p',ascending = False).index

In [9]:
job_index_ordered_by_p

Int64Index([11, 24,  4,  8, 13, 35, 33, 30,  1, 19, 25, 16, 14, 22,  7, 37, 28,
            32, 27, 34,  0, 29, 23, 20, 21, 18, 12,  9,  2,  6, 26,  3, 39, 31,
             5, 36, 17, 15, 10, 38],
           dtype='int64')

In [9]:
#PD算法：任务按照p值排序
pd_job = JOB.copy()

In [10]:
pd_job_after_sequencing,pd_job_index_sequencing_chosen = \
    pd_sequencing(parameter,job_index_ordered_by_p,pd_job)

In [11]:
pd_job_after_timing = \
    pd_timing(pd_job_index_sequencing_chosen,pd_job_after_sequencing)

In [12]:
pd_score = score(pd_job_after_timing,TSF,parameter)

In [13]:
pd_timetable = get_timetable(pd_job_after_timing,parameter)

In [7]:
def pd_get_score(f_parameter, f_job_index_ordered_by_p, f_job, f_tsf):
    f_job_after_sequencing, f_job_index_sequencing_chosen = \
        pd_sequencing(f_parameter, f_job_index_ordered_by_p, f_job)
    f_job_after_timing = \
        pd_timing(f_job_index_sequencing_chosen, f_job_after_sequencing)
    f_score = score(f_job_after_timing, f_tsf, f_parameter)
    return f_score

In [39]:
pd_job = JOB.copy()
pd_get_score(parameter,job_index_ordered_by_p,pd_job,TSF)

686.8555555555556

## LA算法

In [8]:
def LA(job, f_job_index, f_parameter, f_tsf):
    f_job = job.copy()

    #总时间轴,1代表位置可用
    f_timetable = pd.Series(1, index=range(f_parameter['time_units']))
    #任务时间轴,1代表该时间在该任务的窗口中
    empty_timetable = pd.Series(0, index=range(f_parameter['time_units']))

    for i in range(f_parameter['n']):
        job_index = f_job_index[i]
        job_info = f_job.loc[job_index,]

        d_min = job_info.loc['d_min']
        d_max = job_info.loc['d_max']
        w_o = job_info.loc['w_o']
        w_f = job_info.loc['w_f']
        p = job_info.loc['p']
        w = job_info.loc['w']

        test_score = 0
        best_start = 0
        best_stop = 0

        for start in range(w_o,w_f-d_min+1):
            for d in range(d_min,min(d_max,w_f-start)+1):
                stop = start+d
                test_timetable = empty_timetable.copy()
                test_timetable.loc[range(start,stop)]=1

                if np.sum(test_timetable * f_timetable) == d:
                    test_LA_job = f_job.copy()
                    test_LA_job.loc[job_index,'start'] = start
                    test_LA_job.loc[job_index,'stop'] = stop
                    test_job_index_ordered_by_p = f_job_index[(i + 1):]

                    new_test_score = \
                        pd_get_score(f_parameter, test_job_index_ordered_by_p, test_LA_job, f_tsf)

                    test_score = max(test_score,new_test_score)
                    if new_test_score == test_score:
                        best_start = start
                        best_stop = stop

        f_job.loc[job_index, 'start'] = best_start
        f_job.loc[job_index, 'stop'] = best_stop

        f_timetable.loc[range(best_start, best_stop)]=0

    return f_job

In [17]:
LA_job = LA(JOB,job_index_ordered_by_p,parameter,TSF)

In [18]:
LA_score = score(LA_job,TSF,parameter)

In [19]:
LA_timetable = get_timetable(LA_job,parameter)

## GA算法

In [9]:
def PMX_crossover(parent1, parent2):
    '''
    parent1 and parent2 are 1D np.array
    '''
    rng = np.random.default_rng()

    cutoff_1, cutoff_2 = np.sort(rng.choice(np.arange(len(parent1)+1), size=2, replace=False))

    def PMX_one_offspring(p1, p2):
        offspring = np.zeros(len(p1), dtype=p1.dtype)

        # Copy the mapping section (middle) from parent1
        offspring[cutoff_1:cutoff_2] = p1[cutoff_1:cutoff_2]

        # copy the rest from parent2 (provided it's not already there
        for i in np.concatenate([np.arange(0,cutoff_1), np.arange(cutoff_2,len(p1))]):
            candidate = p2[i]
            while candidate in p1[cutoff_1:cutoff_2]: # allows for several successive mappings
                #print(f"Candidate {candidate} not valid in position {i}") # DEBUGONLY
                candidate = p2[np.where(p1 == candidate)[0][0]]
            offspring[i] = candidate
        return offspring

    offspring1 = PMX_one_offspring(parent1, parent2)

    return offspring1


In [21]:
# parent1 = np.array([1,2,3,4,5,6,7,8,9])
# parent2 = np.array([5,4,6,7,2,1,3,9,8])
# son1 = PMX_crossover(parent1, parent2)

In [10]:
def GA(JOB,parameter,TSF):
    GA_all = pd.DataFrame(columns=['index_list','score'])
    GA_job = JOB.copy()
    for i in range(100):
        index_list = np.arange(40)
        np.random.shuffle(index_list)
        GA_score = pd_get_score(parameter,index_list,GA_job,TSF)
        row = {'index_list':index_list,
               'score':GA_score}
        GA_all.loc[i] = row

    for i in range(2000):
        GA_all.sort_values(by='score',ascending = False,inplace=True)
        parent1_index = np.random.randint(50)
        parent2_index = np.random.randint(100)
        GA_all.loc[parent1_index,'index_list']
        son1 = PMX_crossover(GA_all.loc[parent1_index,'index_list'],
                             GA_all.loc[parent2_index,'index_list'])
        GA_new_score = pd_get_score(parameter,son1,GA_job,TSF)
        n = 0
        while sum(GA_all['score'] == GA_new_score)>0 and n < 2:
            index1 = np.random.randint(40)
            index2 = np.random.randint(40)
            mutation1 = son1[index1]
            mutation2 = son1[index2]
            son1[index1] = mutation2
            son1[index2] = mutation1
            GA_new_score = pd_get_score(parameter,son1,GA_job,TSF)
            n += 1

        row = {'index_list':son1,
               'score':GA_new_score}
        GA_all.iloc[99] = row

    GA_score =max(GA_all['score'])

    return GA_score

In [49]:
GA(JOB,parameter,TSF)

815.0

In [ ]:
GA_all = pd.DataFrame(columns=['index_list','Skipped','score'])
GA_job = JOB.copy()

In [ ]:
for i in range(100):
    index_list = np.arange(40)
    np.random.shuffle(index_list)
    GA_score = pd_get_score(parameter, index_list, GA_job, TSF)
    row = {'index_list': index_list,
           'score': GA_score}
    GA_all.loc[i] = row
GA_all.sort_values(by='score', ascending=False, inplace=True)

In [ ]:
for i in range(100):
    GA_all.sort_values(by='score', ascending=False, inplace=True)
    parent1_index = np.random.randint(50)
    parent2_index = np.random.randint(100)
    GA_all.loc[parent1_index, 'index_list']
    son1 = PMX_crossover(GA_all.loc[parent1_index, 'index_list'],
                         GA_all.loc[parent2_index, 'index_list'])
    GA_new_score = pd_get_score(parameter, son1, GA_job, TSF)
    n = 0
    while sum(GA_all['score'] == GA_new_score) > 0 and n < 3:
        index1 = np.random.randint(40)
        index2 = np.random.randint(40)
        mutation1 = son1[index1]
        mutation2 = son1[index2]
        son1[index1] = mutation2
        son1[index2] = mutation1
        GA_new_score = pd_get_score(parameter, son1, GA_job, TSF)
        n += 1

    row = {'index_list': son1,
           'score': GA_new_score}
    GA_all.iloc[99] = row
GA_score = max(GA_all['score'])

## 一次运行


In [11]:
# 设置参数
parameter = {'n': 40,
             'time_units': 100,
             'w_max': 25,
             'd_min': 1,
             'd_max': 25,
             'p_max': 10}

np.random.seed(1)

In [12]:
#初始化
TSF, JOB = initialization(parameter)
job_index_ordered_by_p = JOB.sort_values(by='p', ascending=False).index

In [13]:
job_list = pd.DataFrame(columns=[],
                        dtype = int)

job_list.insert(job_list.shape[1],'list',job_index_ordered_by_p)
job_list.insert(job_list.shape[1],'skip',[1]*40)
job_list.insert(job_list.shape[1],'worst',[1]*40)

In [14]:
job_index_ordered_by_p_1 = list(job_list['list'])

In [15]:
def once():
    pd_job = JOB.copy()
    pd_score = pd_get_score(parameter,job_index_ordered_by_p,pd_job,TSF)

    LA_job = LA(JOB, job_index_ordered_by_p, parameter, TSF)
    LA_score = score(LA_job, TSF, parameter)

    GA_score = GA(JOB,parameter,TSF)

    return pd_score,LA_score,GA_score


In [19]:
pd_job = JOB.copy()
pd_score = pd_get_score(parameter,job_index_ordered_by_p,pd_job,TSF)

In [66]:
new_job_list = job_index_ordered_by_p[20:39]
pd_score = pd_get_score(parameter,new_job_list,pd_job,TSF)

## 增加变异


In [24]:
job = JOB

In [55]:
def pd_sequencing(parameter,job_list,job):
    pd_job = job.copy()
    pd_jobindex = list(job_list['list'])
    pd_skip = list(job_list['skip'])
    pd_worst = list(job_list['worst'])
    #总时间轴,1代表位置可用
    pd_timetable = pd.Series(1,index=range(parameter['time_units']))
    #初始化总时间轴，因为有可能已经有任务指定了时间
    for i in range(parameter['n']):
        pd_timetable.loc[range(pd_job.loc[i,'start'],pd_job.loc[i,'stop'])]=0
    #任务时间轴,1代表该时间在该任务的窗口中
    empty_timetable = pd.Series(0,index=range(parameter['time_units']))
    #建立空工作表，按顺序存放被选择的工作的序号
    chosen_job_list = list()

    #对每一个工作进行判断len(pd_jobindex)
    for i in range(1):

        if pd_skip[i]==0:
            continue

        job_index = pd_jobindex[i]
        job_info = pd_job.loc[job_index,]

        d_min = job_info.loc['d_min']
        d_max = job_info.loc['d_max']
        w_o = job_info.loc['w_o']
        w_f = job_info.loc['w_f']
        p = job_info.loc['p']
        w = job_info.loc['w']
        #下序列用来存放每一个开始时间，长度为d_min的工作带来的收益
        position_value = pd.Series(0,index=range(parameter['time_units']))
        #假设没有可供使用的位置
        at_least_one = False
        #对每一个位置进行测试
        for j in range(w-d_min+1):
            test_start = w_o+j
            test_stop = test_start+d_min
            test_job = empty_timetable.copy()
            test_job.loc[range(test_start,test_stop)]=1
            #判断位置是否可用
            if np.sum(test_job*pd_timetable) == d_min:
                at_least_one = True
                position_value.loc[test_start] = \
                    np.sum(TSF.loc[range(test_start,test_stop),job_index])
        #如果有至少一个可用位置，则选择最大的那个
        if at_least_one:
            # 判断最大最小位置
            if pd_worst[i]==1:
                position_value_bw = position_value.max()
            else:
                position_value_bw = \
                    pd.Series(list(set(position_value))).sort_values()[1]

            decide_start = \
                np.min(position_value.index[position_value == position_value_bw])
            decide_stop = decide_start+d_min
            #更新工作表
            pd_job.loc[job_index,'start'] = decide_start
            pd_job.loc[job_index,'stop'] = decide_stop
            #更新工作序号表
            chosen_job_list.append(job_index)
        else:
            decide_start = 0
            decide_stop = 0
        #刚刚被占用的位置不再可用
        pd_timetable.loc[range(decide_start,decide_stop)]=0

    return pd_job,chosen_job_list